In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense,Dropout,Activation,Conv2D,MaxPooling2D,BatchNormalization,Flatten
from keras.models import Sequential
from keras.optimizers import RMSprop
from keras.callbacks import EarlyStopping,ReduceLROnPlateau,ModelCheckpoint
from keras.models import load_model
import cv2
from PIL import Image
import numpy as np
import pandas as pd
import os
from keras.utils import to_categorical
import seaborn as sns

import scipy
from keras.preprocessing.image import ImageDataGenerator

In [2]:
files_path = "./FinalProjData"

int2emotions = {0:'angry',1:'fear',2:'happy',3:'neutral',4:'sad',5:'surprise',6:'disgust'}
emotions2int = {'angry':0,'fear':1,'happy':2,'neutral':3,'sad':4,'surprise':5,'disgust':6}

dic = {'images':[], 'labels':[], 'purpose':[], 'image_path':[]}

for d in os.listdir(f'{files_path}/fer2013/'):
    print(d)
    for emotion in os.listdir(f'{files_path}/fer2013/{d}'):
        print(emotion)
        for i in os.listdir(f'{files_path}/fer2013/{d}/{emotion}'):
            img = cv2.imread(f'{files_path}/fer2013/{d}/{emotion}/{i}',0)
            img = img.reshape(48,48,1)

            dic['images'].append(img)
            dic['labels'].append(emotion)
            dic['image_path'].append(f'{files_path}/fer2013/{d}/{emotion}/{i}')

            if d=='train':
                dic['purpose'].append('T')
            else:
                dic['purpose'].append('V')

df = pd.DataFrame(dic)
df.head()

train
angry
disgust
fear
happy
neutral
sad
surprise
validation
angry
disgust
fear
happy
neutral
sad
surprise


,images,labels,purpose,image_path
0,"[[[50], [32], [15], [11], [12], [10], [10], [8...",angry,T,./FinalProjData/fer2013/train/angry/Training_1...
1,"[[[29], [24], [29], [11], [15], [22], [26], [1...",angry,T,./FinalProjData/fer2013/train/angry/Training_1...
2,"[[[0], [0], [0], [0], [0], [0], [0], [0], [0],...",angry,T,./FinalProjData/fer2013/train/angry/Training_1...
3,"[[[155], [138], [98], [112], [94], [97], [111]...",angry,T,./FinalProjData/fer2013/train/angry/Training_1...
4,"[[[211], [215], [220], [221], [219], [216], [2...",angry,T,./FinalProjData/fer2013/train/angry/Training_1...


In [3]:
train_data = df[df['purpose']=='T']
val_data = df[df['purpose']=='V']

In [ ]:
train_data.head()

In [ ]:
val_data.head()

In [ ]:
train_data['labels'].value_counts()

In [ ]:
val_data['labels'].value_counts()

In [4]:
happy_df = train_data[train_data['labels']=='happy'].sample(n=3171)
neutral_df = train_data[train_data['labels']=='neutral'].sample(n=3171)
sad_df = train_data[train_data['labels']=='sad'].sample(n=3171)
fear_df = train_data[train_data['labels']=='fear'].sample(n=3171)
angry_df = train_data[train_data['labels']=='angry'].sample(n=3171)
surprise_df = train_data[train_data['labels']=='surprise'].sample(n=3171)
disgust_df = train_data[train_data['labels']=='disgust'].sample(n=436)

train_data = pd.concat([happy_df,neutral_df,sad_df,fear_df,angry_df,surprise_df, disgust_df])

train_data = train_data.sample(frac=1)
train_data.reset_index(inplace=True)
train_data.drop('index',inplace=True,axis=1)

train_data.head()

,images,labels,purpose,image_path
0,"[[[222], [211], [219], [221], [222], [224], [2...",happy,T,./FinalProjData/fer2013/train/happy/Training_7...
1,"[[[255], [255], [255], [255], [255], [251], [2...",happy,T,./FinalProjData/fer2013/train/happy/Training_4...
2,"[[[121], [122], [137], [139], [141], [150], [1...",surprise,T,./FinalProjData/fer2013/train/surprise/Trainin...
3,"[[[8], [18], [26], [24], [22], [15], [12], [25...",neutral,T,./FinalProjData/fer2013/train/neutral/Training...
4,"[[[82], [89], [87], [83], [78], [75], [82], [1...",angry,T,./FinalProjData/fer2013/train/angry/Training_2...


In [ ]:
train_data['labels'].value_counts()

In [ ]:
sns.countplot(train_data['labels'])

In [5]:
batch_size= 32
classes = 7
rows,columns=48,48

In [ ]:
val_data['labels'].value_counts()

In [ ]:
# Convert dataframe to numpy array
train_labels = list(train_data['labels'].replace(emotions2int))
train_labels = to_categorical(train_labels)

val_labels = list(val_data['labels'].replace(emotions2int))
val_labels = to_categorical(val_labels)

train_data = list(train_data['images'])
train_data = np.array(train_data)

val_data = list(val_data['images'])
val_data = np.array(val_data)

In [ ]:
train_data.shape

In [ ]:
val_data.shape

In [6]:
model = Sequential()

# First Block
model.add(Conv2D(64,(3,3),activation='elu',input_shape=(rows,columns,1),kernel_initializer='he_normal',padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3),activation='elu',input_shape=(rows,columns,1),kernel_initializer='he_normal',padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# Second Block
model.add(Conv2D(128,(3,3),activation='elu',kernel_initializer='he_normal',padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3),activation='elu',kernel_initializer='he_normal',padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# Third Block
model.add(Conv2D(256,(3,3),activation='elu',kernel_initializer='he_normal',padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(256,(3,3),activation='elu',kernel_initializer='he_normal',padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# Fourth Block
model.add(Conv2D(512,(3,3),activation='elu',kernel_initializer='he_normal',padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(512,(3,3),activation='elu',kernel_initializer='he_normal',padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# Fifth Block
model.add(Flatten())
model.add(Dense(256,activation='elu',kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Sixth Block
model.add(Dense(128,activation='elu',kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Seventh Block
model.add(Dense(64,activation='elu',kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Eighth Block
model.add(Dense(classes,activation='softmax',kernel_initializer='he_normal'))

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 64)        640       
                                                                 
 batch_normalization (BatchN  (None, 48, 48, 64)       256       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 48, 48, 64)        36928     
                                                                 
 batch_normalization_1 (Batc  (None, 48, 48, 64)       256       
 hNormalization)                                                 
                                                                 
 max_pooling2d (MaxPooling2D  (None, 24, 24, 64)       0         
 )                                                               
                                                        

In [7]:
checkpoint = ModelCheckpoint(f'{files_path}\model\\6_class_emotion_detector_V2.h5',
                             save_best_only=True,
                             mode='min',
                             monitor='val_loss',
                             verbose=1)

earlystopping = EarlyStopping(patience=10,
                             verbose=1,
                             min_delta=0,
                             monitor='val_loss',
                             restore_best_weights=True)


callbacks = [checkpoint, earlystopping]

model.compile(metrics=['accuracy'],
             optimizer='rmsprop',
             loss='categorical_crossentropy')

In [8]:
train_samples = 28273
validation_samples = 3534
batch_size = 64
epochs=30

In [ ]:
#OLD WAY
history = model.fit(train_data,
                    train_labels,
                    epochs=epochs,
                    steps_per_epoch=train_samples//batch_size,
                    validation_data=(val_data,val_labels),
                    validation_steps=validation_samples//batch_size,
                    callbacks=callbacks)

In [9]:
# Data Augmentation setup for training data
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

# Data generator for validation data (No augmentation)
val_datagen = ImageDataGenerator()

# Create generators to read images from dataframe
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_data,
    directory=None,  # Directory is None since paths are absolute
    x_col='image_path',
    y_col='labels',
    target_size=(48, 48),
    color_mode='grayscale',
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True)

validation_generator = val_datagen.flow_from_dataframe(
    dataframe=val_data,
    directory=None,
    x_col='image_path',
    y_col='labels',
    target_size=(48, 48),
    color_mode='grayscale',
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=False)

# Training the model using generators
history = model.fit(
    train_generator,
    steps_per_epoch=train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_samples // batch_size,
    callbacks=callbacks)

Found 19462 validated image filenames belonging to 7 classes.
Found 7178 validated image filenames belonging to 7 classes.
Epoch 1/30
305/441 [===================>..........] - ETA: 4:31 - loss: 2.4222 - accuracy: 0.1669WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 13230 batches). You may need to use the repeat() function when building your dataset.

Epoch 1: val_loss improved from inf to 3.29673, saving model to ./FinalProjData\model\6_class_emotion_detector_V2.h5
441/441 [==============================] - 640s 1s/step - loss: 2.4222 - accuracy: 0.1669 - val_loss: 3.2967 - val_accuracy: 0.1006


In [ ]:
import cv2
from keras.models import load_model
import numpy as np

int2emotions = {0:'angry',1:'fear',2:'happy',3:'neutral',4:'sad',5:'surprise',6:'disgust'}
model = load_model(f'{files_path}\model\\6_class_emotion_detector_V2.h5')

classifier = cv2.CascadeClassifier(f'{files_path}\Haarcascades\\haarcascade_frontalface_default.xml')

def detect_face(frame):
    faces=classifier.detectMultiScale(frame,1.3,4)
    if len(faces) == 0:
        return frame
    for x,y,w,h in faces:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(172,42,251),2)
        face = frame[y:y+h,x:x+w]
        face = cv2.cvtColor(face,cv2.COLOR_BGR2GRAY)
        face = cv2.resize(face,(48,48))
        face = face.reshape(1,48,48,1)
        cv2.putText(frame,text=int2emotions[np.argmax(model.predict(face))],
                    org=(x,y-15),fontFace=cv2.FONT_HERSHEY_SIMPLEX,fontScale=1,color=(106,40,243),thickness=2)
    return frame

In [ ]:
cap = cv2.VideoCapture(0)

while 1:
    ret,frame= cap.read()
    if ret==True:
        cv2.imshow('emotion_detector',detect_face(frame))
        if cv2.waitKey(1)==27:
            break
cap.release()
cv2.destroyAllWindows()